# Prediction of the flow (24 h)

### Decision tree regression
Predicting next 24 hours on the previous predicted value

### Importing necessary libraries

In [243]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor
import datetime as dt
import numpy as np

### Loading the data set

In [244]:
flow_set = pd.read_csv('../dataset/flow1.csv', parse_dates={'hour':['time']},index_col='hour')
dayfirst=True
flow_set = flow_set.resample('60T').mean()

In [245]:
flow_set = flow_set.reset_index()

In [246]:
flow_set['time'] = flow_set['hour']

In [247]:
flow_set['hour'] = (flow_set['hour'].dt.hour)

In [248]:
flow_set.head()

,hour,flow,time
0,14,113.754443,2015-06-01 14:00:00
1,15,113.013333,2015-06-01 15:00:00
2,16,114.081666,2015-06-01 16:00:00
3,17,116.459167,2015-06-01 17:00:00
4,18,123.825000,2015-06-01 18:00:00


### Adding the feature of previous hour's flow value

In [249]:
flow_set['previous-hour'] = flow_set['flow'].shift(1)

In [250]:
flow_set.head()

,hour,flow,time,previous-hour
0,14,113.754443,2015-06-01 14:00:00,NaN
1,15,113.013333,2015-06-01 15:00:00,113.754443
2,16,114.081666,2015-06-01 16:00:00,113.013333
3,17,116.459167,2015-06-01 17:00:00,114.081666
4,18,123.825000,2015-06-01 18:00:00,116.459167


### Dealing with missing values by filling with the mean

In [251]:
flow_set = flow_set.fillna(flow_set.mean())

### Splitting the data set into train and test sets

In [252]:
split_date = pd.datetime(2016,12,31,23,59)

flow_training = flow_set.loc[flow_set['time'] <= split_date]
flow_test = flow_set.loc[flow_set['time'] > split_date]

### Creating training and test variables

In [253]:
X_flow_training = flow_training[['hour','previous-hour']]
X_flow_test = flow_test[['hour','previous-hour','time']]
Y_flow_training = flow_training['flow']
Y_flow_test = flow_test['flow']

### Creating a multiple regression model with decision tree

In [254]:
regressor = DecisionTreeRegressor()
regressor.fit(X_flow_training, Y_flow_training)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

### Predicting next 24 hours on a previous predicted value

In [255]:
X_flow_test['actual-value'] = Y_flow_test

In [256]:
X_flow_test['24-hour'] = X_flow_test['actual-value']

In [257]:
X_flow_test.head()

,hour,previous-hour,time,actual-value,24-hour
13906,0,99.808334,2017-01-01 00:00:00,94.330833,94.330833
13907,1,94.330833,2017-01-01 01:00:00,91.927501,91.927501
13908,2,91.927501,2017-01-01 02:00:00,90.722501,90.722501
13909,3,90.722501,2017-01-01 03:00:00,87.402501,87.402501
13910,4,87.402501,2017-01-01 04:00:00,81.485833,81.485833


### Leaving the 24-hour column with value only for hour 0

In [258]:
X_flow_test['24-hour'].loc[X_flow_test['hour'] != 0] = 0

/home/alicja/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


### Creating the check column with True where hour is 0

In [259]:
check = (X_flow_test['hour'] == 0)
check.head()
X_flow_test['check'] = check
X_flow_test.head()

,hour,previous-hour,time,actual-value,24-hour,check
13906,0,99.808334,2017-01-01 00:00:00,94.330833,94.330833,True
13907,1,94.330833,2017-01-01 01:00:00,91.927501,0.000000,False
13908,2,91.927501,2017-01-01 02:00:00,90.722501,0.000000,False
13909,3,90.722501,2017-01-01 03:00:00,87.402501,0.000000,False
13910,4,87.402501,2017-01-01 04:00:00,81.485833,0.000000,False


### Creating the proper index starting from 0

In [260]:
X_flow_test['index'] = range(0, len(X_flow_test)-1 + 1)

In [261]:
X_flow_test = X_flow_test.set_index('index')

### Creating the formula for next 24 hour prediction
Starting from known value for hour 0

In [262]:
for i in range(len(X_flow_test)-1):
    row = X_flow_test.iloc[[i]]
    next_row = X_flow_test.iloc[[i+1]]
    if row['check'].iloc[0] == True:
        test = row[['hour','actual-value']]
        X_flow_test.loc[i,('24-hour')] = X_flow_test['actual-value'].iloc[0]
        X_flow_test.loc[i+1,('previous-hour')] = X_flow_test.at[i,'24-hour']
    else:
        test = X_flow_test[['hour','previous-hour']].iloc[[i]]
        X_flow_test.loc[i,('24-hour')] = regressor.predict(test)
        X_flow_test.at[i+1,'previous-hour'] = X_flow_test.at[i,'24-hour']

### Showing the results 

In [263]:
X_flow_test.head()

,hour,previous-hour,time,actual-value,24-hour,check
index,,,,,,
0,0,99.808334,2017-01-01 00:00:00,94.330833,94.330833,True
1,1,94.330833,2017-01-01 01:00:00,91.927501,85.107501,False
2,2,85.107501,2017-01-01 02:00:00,90.722501,77.250835,False
3,3,77.250835,2017-01-01 03:00:00,87.402501,73.800000,False
4,4,73.800000,2017-01-01 04:00:00,81.485833,83.626666,False


### Calculating the feature importance and error

In [264]:
feature_importances = pd.DataFrame(regressor.feature_importances_,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
0,0.668177
1,0.331823


### Implementing metrics (mean absolute percentage error)

In [265]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [266]:
mean_absolute_percentage_error(Y_flow_test, X_flow_test['24-hour'])

10.266925472920583